## S05 T01: Transformació Registre Log amb Regular expressions

### Exercici 1

Estandaritza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources". 

In [65]:
import pandas as pd
import numpy as np
import re
import datetime
import warnings
warnings.filterwarnings('ignore')

In [66]:
data_link='/Users/Diana/Downloads/Data-Science-main/Data-sources/Web_access_log-akumenius.com.txt'
pattern=r'\s(?=(?![^\[]*\])(?:[^"]*"[^"]*")*[^"]*$)'
logs = pd.read_csv(
    data_link,
    sep=pattern,
    engine='python',
    na_values='-',
    header=None,
    usecols=[0, 1, 4, 5, 6, 7, 8, 9],
    names=['Web','Ip_address', 'Date/Time','Get','Number','Size','Page','Browser'])
converters={'Number': int,
            'Size': int}

"sep" atribute explanation:

one of our entries:

localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-

we need to:

    split by white space
    split by ""
    split values considered NaN

sep=

r=calling for RegEx

\s(?=split into spaces with these exceptions:

(?![^[]*])=expressions inside brackets. This expression matches any single character not in brackets.

(?:[^"]*"[^"]*")[^"]$)=expressions with double commas. 


In [67]:
logs.head()

,Web,Ip_address,Date/Time,Get,Number,Size,Page,Browser
0,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
1,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
2,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
3,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
4,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""


In [68]:
logs.tail()

,Web,Ip_address,Date/Time,Get,Number,Size,Page,Browser
261868,www.akumenius.com,5.255.253.53,[02/Mar/2014:03:05:39 +0100],"""GET / HTTP/1.1""",200,7528.0,"""-""","""Mozilla/5.0 (compatible; YandexBot/3.0; +http..."
261869,www.akumenius.com,74.86.158.107,[02/Mar/2014:03:09:52 +0100],"""HEAD / HTTP/1.1""",200,NaN,"""-""","""Mozilla/5.0+(compatible; UptimeRobot/2.0; htt..."
261870,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
261871,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
261872,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""


### Exercici 2

Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web. 

In [69]:
#remove double quotes on every single string

logs=logs.replace('"', '', regex=True)

In [70]:
#remove brackets

logs['Date/Time'] =  logs['Date/Time'].apply(lambda x: x.replace('[','').replace(']','')) 
logs.head()

,Web,Ip_address,Date/Time,Get,Number,Size,Page,Browser
0,localhost,127.0.0.1,23/Feb/2014:03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
1,localhost,127.0.0.1,23/Feb/2014:03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
2,localhost,127.0.0.1,23/Feb/2014:03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
3,localhost,127.0.0.1,23/Feb/2014:03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
4,localhost,127.0.0.1,23/Feb/2014:03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)


In [71]:
#I will separate into columns date, time, and time zone
#Insert in columns 3,4,5

logs.insert(3,'Date',logs['Date/Time'].apply(lambda k: k.split(":",1)[0]))
logs.insert(4,'Time',logs['Date/Time'].apply(lambda k: k.split(":",1)[1]))
logs.insert(5,'Time_Zone',logs['Date/Time'].apply(lambda k: k.split(" ",1)[1]))
logs['Time']=logs['Time'].apply(lambda k: k.split(" ",1)[0])

#drop the column 'DATE/TIME'

logs=logs.drop('Date/Time',axis=1)

logs.head()

,Web,Ip_address,Date,Time,Time_Zone,Get,Number,Size,Page,Browser
0,localhost,127.0.0.1,23/Feb/2014,03:10:31,+0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
1,localhost,127.0.0.1,23/Feb/2014,03:10:31,+0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
2,localhost,127.0.0.1,23/Feb/2014,03:10:31,+0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
3,localhost,127.0.0.1,23/Feb/2014,03:10:31,+0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
4,localhost,127.0.0.1,23/Feb/2014,03:10:31,+0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)


In [72]:
#To change the date format so it can be more readable

logs['Date'] = pd.to_datetime(logs['Date'])
logs.head()

,Web,Ip_address,Date,Time,Time_Zone,Get,Number,Size,Page,Browser
0,localhost,127.0.0.1,2014-02-23,03:10:31,+0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
1,localhost,127.0.0.1,2014-02-23,03:10:31,+0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
2,localhost,127.0.0.1,2014-02-23,03:10:31,+0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
3,localhost,127.0.0.1,2014-02-23,03:10:31,+0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
4,localhost,127.0.0.1,2014-02-23,03:10:31,+0100,OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)


### Exercici 3

Geolocalitza les IP's.

In [73]:
#I will extract first all IPs and I will add a column with the number of visits by IP

ip=logs['Ip_address'].value_counts().rename_axis('Ip_address').reset_index(name='Visits')
ip.head()

,Ip_address,Visits
0,66.249.76.216,46382
1,80.28.221.123,14725
2,127.0.0.1,13892
3,217.125.71.222,5201
4,66.249.75.148,3558


In [74]:
#I will get only 25 registers for loading purposes

ip_25= ip[0:26]

ip_25

,Ip_address,Visits
0,66.249.76.216,46382
1,80.28.221.123,14725
2,127.0.0.1,13892
3,217.125.71.222,5201
4,66.249.75.148,3558
5,162.243.192.191,2927
6,62.117.197.230,2567
7,89.128.176.162,1093
8,198.143.133.154,1045
9,176.31.255.177,1044


In [75]:
#Geolocate an IP address using Abstract's geolocation API: https://www.abstractapi.com/guides/how-to-geolocate-an-ip-address-in-python

import json
import requests

In [76]:
#we'll define a geolocate function

def geolocate(ip):
    
    ip_address = ip

   # URL to send the request to
    request_url = 'https://geolocation-db.com/jsonp/' + ip_address
    
    # Send request and decode the result
    response = requests.get(request_url)
    
    try:
        result = response.content.decode()
    
        # Clean the returned string so it just contains the dictionary data for the IP address
        result = result.split("(")[1].strip(")")
    
        # Convert this data into a dictionary
        result  = json.loads(result)
    
    except:
        result = "Error"
    
    return(result)

In [77]:
#Testing the function

#First, select an ip 
random_ip= ip['Ip_address'].iloc[3]
random_ip

'217.125.71.222'

In [78]:
#test de function

geolocate(random_ip)

{'country_code': 'ES',
 'country_name': 'Spain',
 'city': 'Bormujos',
 'postal': '41930',
 'latitude': 37.3736,
 'longitude': -6.0723,
 'IPv4': '217.125.71.222',
 'state': 'Seville'}

In [79]:
#the function works so let's include all the information in the dataframe. I will get only 25 registers for loading purposes

ip_25= ip[0:26]
ip_25

,Ip_address,Visits
0,66.249.76.216,46382
1,80.28.221.123,14725
2,127.0.0.1,13892
3,217.125.71.222,5201
4,66.249.75.148,3558
5,162.243.192.191,2927
6,62.117.197.230,2567
7,89.128.176.162,1093
8,198.143.133.154,1045
9,176.31.255.177,1044


In [80]:
#adding to ip_25 dataframe ip_description from https://geolocation-db.com/jsonp/

ip_25['Ip_description']=ip_25.loc[:,'Ip_address'].apply(lambda k: geolocate(k))

In [81]:
ip_25

,Ip_address,Visits,Ip_description
0,66.249.76.216,46382,"{'country_code': 'US', 'country_name': 'United..."
1,80.28.221.123,14725,"{'country_code': 'ES', 'country_name': 'Spain'..."
2,127.0.0.1,13892,"{'country_code': 'Not found', 'country_name': ..."
3,217.125.71.222,5201,"{'country_code': 'ES', 'country_name': 'Spain'..."
4,66.249.75.148,3558,"{'country_code': 'US', 'country_name': 'United..."
5,162.243.192.191,2927,"{'country_code': 'US', 'country_name': 'United..."
6,62.117.197.230,2567,"{'country_code': 'ES', 'country_name': 'Spain'..."
7,89.128.176.162,1093,"{'country_code': 'ES', 'country_name': 'Spain'..."
8,198.143.133.154,1045,"{'country_code': 'US', 'country_name': 'United..."
9,176.31.255.177,1044,"{'country_code': 'FR', 'country_name': 'France..."


In [82]:
#expand the column 'IP DESCRIPTION'. As the values are a dictionary we will use the function pd.Series and will concatenate
#the two tables

ip_25 = pd.concat([ip_25, ip_25['Ip_description'].apply(pd.Series)], axis=1)

ip_25

,Ip_address,Visits,Ip_description,country_code,country_name,city,postal,latitude,longitude,IPv4,state
0,66.249.76.216,46382,"{'country_code': 'US', 'country_name': 'United...",US,United States,None,None,37.751,-97.822,66.249.76.216,None
1,80.28.221.123,14725,"{'country_code': 'ES', 'country_name': 'Spain'...",ES,Spain,Madrid,28045,40.4165,-3.7026,80.28.221.123,Madrid
2,127.0.0.1,13892,"{'country_code': 'Not found', 'country_name': ...",Not found,Not found,Not found,Not found,Not found,Not found,IP Not found,Not found
3,217.125.71.222,5201,"{'country_code': 'ES', 'country_name': 'Spain'...",ES,Spain,Bormujos,41930,37.3736,-6.0723,217.125.71.222,Seville
4,66.249.75.148,3558,"{'country_code': 'US', 'country_name': 'United...",US,United States,None,None,37.751,-97.822,66.249.75.148,None
5,162.243.192.191,2927,"{'country_code': 'US', 'country_name': 'United...",US,United States,New York,10011,40.7308,-73.9975,162.243.192.191,New York
6,62.117.197.230,2567,"{'country_code': 'ES', 'country_name': 'Spain'...",ES,Spain,Valencia,46015,39.4698,-0.3774,62.117.197.230,Valencia
7,89.128.176.162,1093,"{'country_code': 'ES', 'country_name': 'Spain'...",ES,Spain,Marbella,29602,36.5154,-4.8858,89.128.176.162,Malaga
8,198.143.133.154,1045,"{'country_code': 'US', 'country_name': 'United...",US,United States,Chicago,60604,41.8785,-87.633,198.143.133.154,Illinois
9,176.31.255.177,1044,"{'country_code': 'FR', 'country_name': 'France...",FR,France,None,None,48.8582,2.3387,176.31.255.177,None


In [83]:
ip_25.columns

Index(['Ip_address', 'Visits', 'Ip_description', 'country_code',
       'country_name', 'city', 'postal', 'latitude', 'longitude', 'IPv4',
       'state'],
      dtype='object')

In [84]:
#drop the columns that are not useful

ip_25=ip_25.drop(['IPv4','Ip_description'],axis=1)
ip_25

,Ip_address,Visits,country_code,country_name,city,postal,latitude,longitude,state
0,66.249.76.216,46382,US,United States,None,None,37.751,-97.822,None
1,80.28.221.123,14725,ES,Spain,Madrid,28045,40.4165,-3.7026,Madrid
2,127.0.0.1,13892,Not found,Not found,Not found,Not found,Not found,Not found,Not found
3,217.125.71.222,5201,ES,Spain,Bormujos,41930,37.3736,-6.0723,Seville
4,66.249.75.148,3558,US,United States,None,None,37.751,-97.822,None
5,162.243.192.191,2927,US,United States,New York,10011,40.7308,-73.9975,New York
6,62.117.197.230,2567,ES,Spain,Valencia,46015,39.4698,-0.3774,Valencia
7,89.128.176.162,1093,ES,Spain,Marbella,29602,36.5154,-4.8858,Malaga
8,198.143.133.154,1045,US,United States,Chicago,60604,41.8785,-87.633,Illinois
9,176.31.255.177,1044,FR,France,None,None,48.8582,2.3387,None


### Nivell 3

Mostra'm la teva creativitat, Sorprèn-me fes un pas més enllà amb l'anàlisi anterior.

In [89]:
#I will map an ip_address

#I will choose first the IP:

city=ip_25['city'].iloc[5]
ip=ip_25['Ip_address'].iloc[5]
lat= ip_25['latitude'].iloc[5]
long=ip_25['longitude'].iloc[5]

city,ip,lat,long

('New York', '162.243.192.191', 40.7308, -73.9975)

In [86]:
#import folium for mapping

import folium

In [87]:
#locate the IP address on the map

location = (lat,long)

map = folium.Map(location=location, zoom_start=10)
folium.CircleMarker(location=location, radius=50, color="red").add_to(map)
folium.Marker(location).add_to(map)

map